# Install/ import libraries

In [ ]:
!pip install sentence-transformers

In [15]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from sentence_transformers.cross_encoder import CrossEncoder
import pickle
from scipy import spatial
import os
print(os.getcwd())

C:\Users\yiehy\OneDrive\Desktop\cs425-nlc-project\5.Reranker


# Best performing bi-encoder (retrieve and re-rank)

In [16]:
bi_encoder_model = SentenceTransformer("../8.Fine-tuned Models/finetuned-bertbase-1epoch")
cross_encoder_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2')

# On test set

In [17]:
# Read embeddings
test_answer_embeddings = pickle.load(open("../4.Retrieval/finetuned_bertbase/finetuned_bertbase_test_answer_embeddings.pkl", 'rb'))

In [18]:
test_df = pd.read_csv("../0.Datasets/train_test_split/test.csv")

In [19]:
question_answer_index_map = {}
for _,row in test_df.iterrows():
    if row["qid"] not in question_answer_index_map:
        question_answer_index_map[row["qid"]]= []
        question_answer_index_map[row["qid"]].append(row["docid"])
    else:
        question_answer_index_map[row["qid"]].append(row["docid"])

In [20]:
labels = []
for v in question_answer_index_map.values():
    labels.append(v)

In [21]:
question_map = {}
label_map = {}
for _,row in test_df.iterrows():
    if row["qid"] not in question_map:
        question_map[row["qid"]] = row["question"]
    if row["answer"] not in label_map:
        label_map[row["answer"]] = row["docid"]

In [22]:
test_answer_list = test_df["answer"].tolist()
predictions = []
count=1
for k,v in question_map.items():
    if count%100==0:
        print(count)
    question_embedding = bi_encoder_model.encode(v)
    answer_similiarity = {}
    for i,embed in enumerate(test_answer_embeddings):
        answer_similiarity[i]= np.dot(question_embedding, embed)
    answer_similiarity = {k: v for k, v in sorted(answer_similiarity.items(), key=lambda item: item[1], reverse=True)}
    top_20_hits = []
    for item in list(answer_similiarity)[:20]:
        top_20_hits.append(test_answer_list[item])
    cross_encoder_answer_similiarity = {}
    for hit in top_20_hits:
        cross_encoder_answer_similiarity[hit] = cross_encoder_model.predict([v,hit])
    cross_encoder_answer_similiarity = {k: v for k, v in sorted(cross_encoder_answer_similiarity.items(), key=lambda item: item[1], reverse=True)}
    label_index = label_map[list(cross_encoder_answer_similiarity.keys())[0]]
    predictions.append([label_index])
    count+=1   

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500


In [23]:
#save prediction and results
results = {"labels":labels,"predictions":predictions}
with open("../7.Evaluate/cross_encoder.pkl", 'wb') as f:
    pickle.dump(results, f, pickle.HIGHEST_PROTOCOL)